# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# grab output from the WeatherPy
weather_results_df = pd.read_csv('../WeatherPy/output_data/cities.csv').reset_index(drop=True)
weather_results_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hambantota,40,LK,1580012509,83,6.12,81.12,27.00,6.70
1,1,Cherskiy,100,RU,1580012509,96,68.75,161.30,-27.74,1.21
2,2,Mar del Plata,0,AR,1580012509,76,-38.00,-57.56,17.78,3.41
3,3,Ushuaia,75,AR,1580012405,81,-54.80,-68.30,7.00,11.80
4,4,Chokurdakh,100,RU,1580012510,86,70.63,147.92,-30.13,2.68
...,...,...,...,...,...,...,...,...,...,...
505,505,Yining,100,CN,1580012315,95,29.04,114.56,5.22,0.86
506,506,Brainerd,90,US,1580012343,85,46.36,-94.20,-3.00,0.81
507,507,Usinsk,91,RU,1580012575,100,65.99,57.53,-23.87,0.49
508,508,Nawa,0,IN,1580012575,46,27.02,75.00,14.12,0.90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# store lat and lng in locations and put the humidity column into a veriable
locations = weather_results_df[['Lat', 'Lng']]
humidity = weather_results_df['Humidity']

In [4]:
# plot heatmap
fig = gmaps.figure()

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = max(humidity), point_radius=1)

# add layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Start narrowing down the results by weather conditions that I like
ideal_weather_df = (weather_results_df[weather_results_df['Max Temp']>24]).reset_index(drop=True)
ideal_weather_df = ideal_weather_df[ideal_weather_df['Wind Speed']<10]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness']==0]
ideal_weather_df = ideal_weather_df.dropna(axis=1)
ideal_weather_df = ideal_weather_df.sort_values(by=['Humidity'], ascending=False)

# cut off my list to be a maximum of 10 locations
if len(ideal_weather_df['City'])>10:
    ideal_weather_df = ideal_weather_df.head(10)
ideal_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
67,277,Marawi,0,PH,1580012287,84,8.00,124.29,28.33,3.58
6,17,Rikitea,0,PF,1580012232,77,-23.12,-134.97,26.17,4.57
84,328,Siocon,0,PH,1580012551,77,7.71,122.14,27.70,3.96
24,82,Cockburn Town,0,TC,1580012518,74,21.46,-71.14,24.66,5.51
77,312,Karratha,0,AU,1580012297,71,-20.74,116.85,29.79,4.26
13,41,Pati,0,ID,1580012514,70,-6.76,111.04,29.46,2.64
15,45,Les Cayes,0,HT,1580012514,70,18.20,-73.75,25.46,4.47
31,119,Buala,0,SB,1580012523,59,-8.14,159.59,31.43,0.93
86,334,Panguna,0,PG,1580012552,56,-6.32,155.48,29.69,3.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [6]:
# store the results into a new variable and add a few columns
hotel_df = pd.DataFrame(ideal_weather_df).reset_index(drop=True)
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""
hotel_df = hotel_df.reset_index()
hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,277,Marawi,0,,1580012287,84,8.00,124.29,28.33,3.58,
1,1,17,Rikitea,0,,1580012232,77,-23.12,-134.97,26.17,4.57,
2,2,328,Siocon,0,,1580012551,77,7.71,122.14,27.70,3.96,
3,3,82,Cockburn Town,0,,1580012518,74,21.46,-71.14,24.66,5.51,
4,4,312,Karratha,0,,1580012297,71,-20.74,116.85,29.79,4.26,
5,5,41,Pati,0,,1580012514,70,-6.76,111.04,29.46,2.64,
6,6,45,Les Cayes,0,,1580012514,70,18.20,-73.75,25.46,4.47,
7,7,119,Buala,0,,1580012523,59,-8.14,159.59,31.43,0.93,
8,8,334,Panguna,0,,1580012552,56,-6.32,155.48,29.69,3.12,


In [7]:
# create variables for the radius and target type
target_radius = 5000
target_type = "lodging"
    
# do the api call and put the results into the newly created columns
for index, row in hotel_df.iterrows():
    lat_lng = f"{hotel_df.iloc[index]['Lat']},{hotel_df.iloc[index]['Lng']}"
    params={
            "location": lat_lng,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(base_url,params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Country'] = results[0]['vicinity']
    except:
        pass

hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,277,Marawi,0,"Sultan Omar Dianalan Boulevard, Marawi City",1580012287,84,8.00,124.29,28.33,3.58,Derogongan Residence
1,1,17,Rikitea,0,Rikitea,1580012232,77,-23.12,-134.97,26.17,4.57,Pension Maro'i
2,2,328,Siocon,0,"388 poblacion, Siocon",1580012551,77,7.71,122.14,27.70,3.96,ABS Hotel
3,3,82,Cockburn Town,0,"1 Duke Street, Cockburn Town",1580012518,74,21.46,-71.14,24.66,5.51,Osprey Beach Hotel
4,4,312,Karratha,0,"35/45 Searipple Road, Karratha",1580012297,71,-20.74,116.85,29.79,4.26,ibis Styles Karratha
5,5,41,Pati,0,"117, Jalan Kol. Sunandar No.117, Ngagul, Winong",1580012514,70,-6.76,111.04,29.46,2.64,SPOT ON 2865 Sartika Inn Pati
6,6,45,Les Cayes,0,"Longuefosse, Place de la Concorde #6, Arrondis...",1580012514,70,18.20,-73.75,25.46,4.47,Villa Orphee
7,7,119,Buala,0,Solomon Islands,1580012523,59,-8.14,159.59,31.43,0.93,Maringe Lagoon Lodge
8,8,334,Panguna,0,,1580012552,56,-6.32,155.48,29.69,3.12,


In [8]:
# create the map layout
coordinates = list(zip(hotel_df['Lat'],hotel_df['Lng']))
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df[['Hotel Name', 'City', 'Country']].iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer on map
# fig2 = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…